
"""
This is a more complex example on performing clustering on large scale dataset.
This examples find in a large set of sentences local communities, i.e., groups of sentences that are highly
similar. You can freely configure the threshold what is considered as similar. A high threshold will
only find extremely similar sentences, a lower threshold will find more sentence that are less similar.
A second parameter is 'min_community_size': Only communities with at least a certain number of sentences will be returned.
The method for finding the communities is extremely fast, for clustering 50k sentences it requires only 5 seconds (plus embedding comuptation).
In this example, we download a large set of questions from Quora and then find similar questions in this set.
"""

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=c5f9da9997399787efb4252fbbb21c8fbd831e4a769868f333ce3872a02585a5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import pandas as pd

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
# Get all unique sentences from the file

corpus_sentences = set()

with open("./drive/MyDrive/quora_duplicate_questions.tsv",encoding="utf8") as f:
  reader = csv.DictReader(f,delimiter="\t",quoting=csv.QUOTE_MINIMAL)
  for row in reader:
    corpus_sentences.add(row['question1'])
    corpus_sentences.add(row['question2'])
    if len(corpus_sentences) >= 50000:
      break

In [5]:
corpus_sentences = list(corpus_sentences)
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

In [6]:
print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Start clustering
Clustering done after 3.87 sec


In [8]:
len(clusters)

61

In [7]:
#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])


Cluster 1, #103 Elements 
	 What is the best way of improving spoken English?
	 How can I speak English more fluently?
	 How can I improve my communication skill and English proficiency?
	 ...
	 What is the best way or resources to learn english like a native speakers?
	 How could I improve my English?
	 How can I improve my communication in English?

Cluster 2, #86 Elements 
	 How can I make money online in India?
	 How can I start earning money online ?
	 How can an apprentice programmer make money online?
	 ...
	 How can one make money online?
	 How can I make money online for job?
	 What is best way to make money online?

Cluster 3, #82 Elements 
	 What are the economic implications of banning 500 and 1000 rupee notes?
	 What are your views on banning 500 and 1000 rupee notes? How does it affect black money and is it really gonna work and expose all the black money?
	 What are the advantages of the decision taken by the Government of India to scrap out 500 and 1000 rupees notes?
	